In [1]:
from cost_param import generate_cost_function_params
import os
from animation import readMat, animate_gantry_system
import numpy as np

generate_cost_function_params(6157,3830)

a=33.0, b=29.0


The cost function is: cost = 33*theta_max + 29*x_task

In [2]:
os.chdir("BIP_PID_PACKAGE.GantryTrolley_PID_CONTROL_LOOP")

In [3]:
def single_simulation(Kp = 0, Kd = 0, Ki = 0):
    simulation_command = "./GantryTrolley_PID_CONTROL_LOOP -override Kp="+str(Kp)+",Ki="+str(Ki)+",Kd="+str(Kd)
    os.system(simulation_command)
    [names, data] = readMat("GantryTrolley_PID_CONTROL_LOOP_res.mat")
    return [names, data]

loops = []
loop_dict = dict()
for kp in np.arange(1, 40, 1):
    for kd in np.arange(10, 500, 1):
        [names, data] = single_simulation(kp, kd)
        time_index = names.index("time")
        x_index = names.index("gantryTrolley_PlantModel_Block.x")
        theta_index = names.index("gantryTrolley_PlantModel_Block.theta")
        time_array = data[time_index]
        x_array = data[x_index]
        theta_array = data[theta_index]
        loop_dict["time_array"] = time_array
        loop_dict["x_array"] = x_array
        loop_dict["theta_array"] = theta_array

        loops.append(loop_dict)
        


LOG_STDOUT        | warning | You are overriding Ki with a small value or zero.
|                 | |       | This could lead to numerically dirty solutions or divisions by zero if not tearingStrictness=veryStrict.
LOG_SUCCESS       | info    | The initialization finished successfully without homotopy method.
LOG_SUCCESS       | info    | The simulation finished successfully.
LOG_STDOUT        | warning | You are overriding Ki with a small value or zero.
|                 | |       | This could lead to numerically dirty solutions or divisions by zero if not tearingStrictness=veryStrict.
LOG_SUCCESS       | info    | The initialization finished successfully without homotopy method.
LOG_SUCCESS       | info    | The simulation finished successfully.
LOG_STDOUT        | warning | You are overriding Ki with a small value or zero.
|                 | |       | This could lead to numerically dirty solutions or divisions by zero if not tearingStrictness=veryStrict.
LOG_SUCCESS       | info   

In [7]:
import pickle

with open('objs.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(loops, f)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

def calculate_cost(theta_max, t_task, a, b):
    return a * theta_max + b * t_task

def calculate_t_task(x, theta, time):
    # Find the time at which the trolley reaches the target
    x_target = 20
    x_diff = np.abs(x - x_target)
    t_task = time[np.where(x_diff < 0.01)[0][0]]
    return t_task

def simulate_control(Kp, Kd):
    simulation_command = (
    f"./GantryTrolley_PID_CONTROL_LOOP-override "
    f"GantryTrolley_PID_CONTROL_LOOP.Kp={Kp},"
    f"GantryTrolley_PID_CONTROL_LOOP.Ki=0,"
    f"GantryTrolley_PID_CONTROL_LOOP.Kd={Kd}"
    )
    os.system(simulation_command)
    
    # Load simulation results
    [names, data] = readMat("GantryTrolley_PID_CONTROL_LOOP_res.mat")
    theta_index = names.index("gantryTrolley_PlantModel_Block.theta")
    x_index = names.index("gantryTrolley_PlantModel_Block.x_output")
    time_index = names.index("time")
    theta = data[theta_index]
    x = data[x_index]
    time = data[time_index]
    
    # Calculate cost components
    theta_max = max(abs(np.rad2deg(theta))) # Convert radians to degrees
    t_task = calculate_t_task(x, theta, time) # TODO: Implement calculate_t_task
    return theta_max, t_task

# Sweep over Kp and Kd
a, b = 33, 29 # From cost function parameter calculation
best_cost = float('inf')
best_params = None
for Kp in range(1, 41):
    for Kd in range(10, 501, 10):
        theta_max, t_task = simulate_control(Kp, Kd)
        cost = calculate_cost(theta_max, t_task, a, b)
        if cost < best_cost:
            best_cost = cost
            best_params = (Kp, Kd)
            
print(f"Optimal Parameters: Kp={best_params[0]}, Kd={best_params[1]}, Cost={best_cost}")